In [1]:
import json
import pandas as pd

repo_profiles = {}
with open('repo_profiles.json') as rj:
    for rl in rj.readlines():
        line = rl.split('\t')
        repo = line[0]
        profile = json.loads(line[1])
        repo_profiles[repo] = profile
    
team_profiles = []
teams = []
with open('team_profiles.json') as tj:
    for tl in tj.readlines():
        line = tl.split('\t')
        team = json.loads(line[0])
        profile = json.loads(line[1])
        team_profiles.append(profile)
        teams.append(team)

In [2]:
repo_profiles_df = pd.DataFrame(repo_profiles).transpose()
repo_profiles_df.fillna('',inplace=True)
team_profiles_df = pd.DataFrame(team_profiles,index=[str(t) for t in teams])
team_profiles_df.rename(columns={
                                   'repo_size':'size',
                                   'repo_forks':'forks',
                                   'repo_subscribers':'subscribers',
                                   'repo_watchers':'watchers',
                                   'languages':'languages',
                                   'topics':'topics'
                               },inplace=True)
numerics = ['size','forks','subscribers','watchers']
min = repo_profiles_df[numerics].min()
max = repo_profiles_df[numerics].max()
repo_profiles_df[numerics] = (repo_profiles_df[numerics]-min)/(max-min)
team_profiles_df[numerics] = (team_profiles_df[numerics]-min)/(max-min)

print(repo_profiles_df.iloc[0])
print(team_profiles_df.iloc[0])

forks                                                2.60891e-05
languages                                                       
size                                                   9.597e-05
subscribers                                            0.0037357
topics         [oxygenxml, git, oxygen-standalone-plugin, add...
watchers                                             8.08912e-05
Name: oxygenxml/oxygen-git-plugin, dtype: object
languages      [HTML, CSS, Python, JavaScript]
forks                              0.000122619
size                               2.99895e-05
subscribers                         0.00192622
watchers                           0.000128078
topics                                      []
Name: ['AMorporkian', 'kharidiron'], dtype: object


In [3]:
repo_profiles_df.describe()

,forks,size,subscribers,watchers
count,181323.000000,181323.000000,181323.000000,181323.000000
mean,0.000539,0.001451,0.004377,0.001476
std,0.003829,0.010532,0.015398,0.008520
min,0.000000,0.000000,0.000000,0.000000
25%,0.000005,0.000011,0.000350,0.000000
50%,0.000078,0.000072,0.001401,0.000044
75%,0.000297,0.000483,0.003736,0.000509
max,1.000000,1.000000,1.000000,1.000000


In [4]:
team_profiles_df.describe()

,forks,size,subscribers,watchers
count,22330.000000,2.233000e+04,22330.000000,22330.000000
mean,0.001999,1.710201e-03,0.011510,0.004689
std,0.004260,4.203524e-03,0.020819,0.014758
min,0.000000,3.429433e-07,0.000000,0.000000
25%,0.000245,1.999221e-04,0.002815,0.000240
50%,0.000789,6.011164e-04,0.005915,0.001209
75%,0.002131,1.573824e-03,0.012524,0.004808
max,0.143161,1.714984e-01,0.736896,0.750063


In [5]:
import numpy as np

def euclidean_distance(p1,p2):
    distance = np.linalg.norm(p1[numerics]-p2[numerics])**2
    langs = set(p1['languages']).union(p2['languages'])
    topics = set(p1['topics']).union(p2['topics'])
    p1_langs = pd.Series([lang in p1['languages'] for lang in langs])/np.sqrt(len(langs))
    p2_langs = pd.Series([lang in p2['languages'] for lang in langs])/np.sqrt(len(langs))
    p1_topics = pd.Series([topic in p1['topics'] for topic in topics])/np.sqrt(len(topics))
    p2_topics = pd.Series([topic in p2['topics'] for topic in topics])/np.sqrt(len(topics))
    distance += np.linalg.norm(p1_langs-p2_langs)**2 + np.linalg.norm(p1_topics-p2_topics)**2
    distance = np.sqrt(distance)
    
    return distance
    
p1 = repo_profiles_df.iloc[0]
p2 = team_profiles_df.iloc[0]
print(euclidean_distance(p1,p2))

1.4142147256033752


In [8]:
from queue import PriorityQueue

team_recommendations = []
cnt = 0
for repo,repo_profile in repo_profiles_df.iterrows():
    cnt += 1
    if cnt > 10:
        break
    print(cnt)
    rec = []
    queue = PriorityQueue()
    for team,team_profile in team_profiles_df.iterrows():
        dis = euclidean_distance(repo_profile,team_profile)
        queue.put_nowait((-dis,team))
        if queue.qsize() > 20:
            queue.get_nowait()
    while queue.qsize()>0:
        rec.append(queue.get_nowait())
    team_recommendations.append(rec)


1
2
3
4
5
6
7
8
9
10
11
[[(-1.0000000073497037, "['tspenov', 'LucoLe']"), (-1.0000000056802527, "['tiagoshibata', 'ppathan', 'spryor', 'avanikp', 'mechaffin', 'isaac73', 'hathind-ms']"), (-1.0000000038092847, "['JoanClaret', 'PablitoGS', 'nguasch', 'beatrizip', 'siscu']"), (-0.9929359497256668, "['kivervinicius', 'shadowmaru', 'alinebastos', 'khaosdoctor', 'WendellAdriel', 'wilcorrea', 'angeliski', 'mikaelhadler', 'adeonir', 'giovannicruz97', 'lbighetti', 'lflimeira', 'kellfluz', 'Diego-Brocanelli', 'pacheco7']"), (-0.9746798367656732, "['Hx66', 'mkruijt', 'Razpudding', 'ChrisOwen101', 'fversnel', 'eidosam']"), (-0.9661029323122489, "['MartinHelmut', 'kotzendekrabbe', 'Alexpeschel']"), (-0.9636280149061522, "['arjunkharbanda', '2jiwon', 'SamirJouni', 'ktran031', 'dimitristsaknakis']"), (-0.963625899760768, "['rabinrai44', 'aneagoie', 'thisisabdus', 'drood87', 'MattCSmith', 'juliangilquin', 'skylar01', 'tonydc1997']"), (-0.9607725975606737, "['alexanmtz', 'mthais', 'lucianojfj']"), (-0.

In [13]:
for i,rec in enumerate(team_recommendations):
    print(repo_profiles_df.iloc[i].name,end='\t')
    for tm in rec:
        print(tm,end='\t')
    print('\n')

oxygenxml/oxygen-git-plugin	(-1.0000000073497037, "['tspenov', 'LucoLe']")	(-1.0000000056802527, "['tiagoshibata', 'ppathan', 'spryor', 'avanikp', 'mechaffin', 'isaac73', 'hathind-ms']")	(-1.0000000038092847, "['JoanClaret', 'PablitoGS', 'nguasch', 'beatrizip', 'siscu']")	(-0.9929359497256668, "['kivervinicius', 'shadowmaru', 'alinebastos', 'khaosdoctor', 'WendellAdriel', 'wilcorrea', 'angeliski', 'mikaelhadler', 'adeonir', 'giovannicruz97', 'lbighetti', 'lflimeira', 'kellfluz', 'Diego-Brocanelli', 'pacheco7']")	(-0.9746798367656732, "['Hx66', 'mkruijt', 'Razpudding', 'ChrisOwen101', 'fversnel', 'eidosam']")	(-0.9661029323122489, "['MartinHelmut', 'kotzendekrabbe', 'Alexpeschel']")	(-0.9636280149061522, "['arjunkharbanda', '2jiwon', 'SamirJouni', 'ktran031', 'dimitristsaknakis']")	(-0.963625899760768, "['rabinrai44', 'aneagoie', 'thisisabdus', 'drood87', 'MattCSmith', 'juliangilquin', 'skylar01', 'tonydc1997']")	(-0.9607725975606737, "['alexanmtz', 'mthais', 'lucianojfj']")	(-0.9607694

In [ ]:
"""
oxygenxml/oxygen-git-plugin	(-1.0000000073497037, "['tspenov', 'LucoLe']")	(-1.0000000056802527, "['tiagoshibata', 'ppathan', 'spryor', 'avanikp', 'mechaffin', 'isaac73', 'hathind-ms']")	(-1.0000000038092847, "['JoanClaret', 'PablitoGS', 'nguasch', 'beatrizip', 'siscu']")	(-0.9929359497256668, "['kivervinicius', 'shadowmaru', 'alinebastos', 'khaosdoctor', 'WendellAdriel', 'wilcorrea', 'angeliski', 'mikaelhadler', 'adeonir', 'giovannicruz97', 'lbighetti', 'lflimeira', 'kellfluz', 'Diego-Brocanelli', 'pacheco7']")	(-0.9746798367656732, "['Hx66', 'mkruijt', 'Razpudding', 'ChrisOwen101', 'fversnel', 'eidosam']")	(-0.9661029323122489, "['MartinHelmut', 'kotzendekrabbe', 'Alexpeschel']")	(-0.9636280149061522, "['arjunkharbanda', '2jiwon', 'SamirJouni', 'ktran031', 'dimitristsaknakis']")	(-0.963625899760768, "['rabinrai44', 'aneagoie', 'thisisabdus', 'drood87', 'MattCSmith', 'juliangilquin', 'skylar01', 'tonydc1997']")	(-0.9607725975606737, "['alexanmtz', 'mthais', 'lucianojfj']")	(-0.9607694047726144, "['tedchamb', 'wnjenkin']")	(-0.9538799615486232, "['outofambit', 'aaronshekey']")	(-0.9534660238867818, "['piedcipher', 'kkdroidgit', 'OjasviBhargava', 'vaishnavi-janardhan', 'sabdar18', 'kartikohri1712', 'Yashsonisoni', 'lifeLessCoder', 'ravishankar97']")	(-0.9486848727252053, "['tkaria', 'daanaerts']")	(-0.9258719464434662, "['FraGoTe', 'arslanbilal', 'wzhishen', 'awesome-bot', 'spyshiv', 'parrainc', 'biyiklioglu']")	(-0.9258261506608181, "['y-zaky', 'mineshmshah']")	(-0.9258232920182698, "['eliascodes', 'AseelM', 'BartBucknill', 'AbdullahChaudhry']")	(-0.9258225759959695, "['jsms90', 'rebecacalvoquintero', 'rachaelcodes', 'ameliejyc', 'astroash', 'Jen-Harris', 'azayneeva', 'maxgerber', 'dangerdak', 'shannonjensen']")	(-0.925821666457658, "['m4v15', 'macintoshhelper', 'Heathercoraje', 'sajedazoabi', 'matthewdking', 'KatiaAsh', 'rinoma', 'idantb101', 'Amirk390', 'lucymk', 'vlbee']")	(-0.925821417749809, "['M3kH', 'remarcmij', 'malnajar', 'wouterkleijn']")	(-0.9258213059959588, "['mattlub', 'JWLD', 'm4v15', 'Suuuuha', 'philawsophizing', 'RQ16', 'HilbertSpitzer', 'Karyum', 'MohamedOmarii']")	

yang163email/LearningNotes	(-0.00018899800976598767, "['pengchengan', 'jjamay']")	(-0.00018522326953321995, "['jemaf', 'arthurbrito99', 'wendervitor', 'raul4247', 'gustavo-depaula']")	(-0.00017958766937252273, "['wisegb', 'graichen']")	(-0.00017829623644781946, "['zactimeinc', 'cjmurphy42']")	(-0.00017820634882333427, "['samfalconer', 'asmasattar', 'Jeremy-Friedman', 'amk5401']")	(-0.00017745188394865707, "['egyptianbman', 'renuzit']")	(-0.000175660766263038, "['viviant100', 'vickymyers', 'loriNakamura']")	(-0.0001713788131742423, "['Gaohaoyang', 'TangMingliang95']")	(-0.00016275243724183232, "['zenghou', 'HanYaodong']")	(-0.00014464688167234956, "['franciscofranco', 'anton0kurilov']")	(-0.00012675267046170793, "['jluangphasy', 'cmd-kvn']")	(-0.0001228403454719345, "['brianlmacdonald', 'jvpepino', 'apdjz']")	(-0.00012029555501961326, "['janie-t', 'anna-marie-vos', 'lucas-wills', 'MeghanHughes']")	(-0.00011944173961541404, "['mtferreira', 'lucasinfoh']")	(-0.00011819204596219795, "['dlozanoarenal', 'rcarballoc']")	(-0.00011785764048331526, "['Mortymus', 'ingokru']")	(-0.00011785764048331526, "['mmoscher', 'da-snap']")	(-0.00011697013947334562, "['subdavis', 'cconsidine']")	(-8.286283469993914e-05, "['Beeert', 'WhiteWolf3500']")	(-5.853082562850996e-05, "['jiexray', '18822265922']")	

swift-nav/librtcm	(-0.7453598015230589, "['casallas', 'rpavlik', 'Oberon00', 'vdudouyt', 'janoc', 'sradigan', 'cpldcpu']")	(-0.7453586676325212, "['kimmoli', 'sledges', 'Nokius', 'martinbrook', 'alinelena', 'krnlyng', 'mlehtima', 'ballock']")	(-0.7071756543834669, "['vuvova', 'kevgs', 'svoj', 'elenst', 'midenok', 'knielsen', 'dr-m', 'abarkov', 'sanja-byelkin', 'montywi', 'dbart']")	(-0.7071592244059751, "['timthirion', 'bastienjacquet']")	(-0.7071524937103597, "['fumieval', 'aavogt', 'jstolarek']")	(-0.707115380306641, "['pinotree', 'HenryMiller1']")	(-0.707114293229157, "['sumeet-jain-engineer', 'andrew-elder', 'grandcat', 'intel-ethernet', 'pinealservo', 'davidcemin', 'jdkoftinoff', 'MarcinMiklas', 'christopher-s-hall', 'kencarlino', 'RikusW', 'cirust20', 'mprinn', 'k-gupta', 'rhii', 'bdthomsen', 'jfornal', 'daejungkim', 'PawelModrzejewski', 'vvacharya', 'aditpape', 'freddeng', 'jongwook-lee']")	(-0.7071110566189798, "['a-ilango', 'sydidelot', 'JoZie', 'aingerson', 'dmitrygladkov', 'vkrishna', 'fzago-cray']")	(-0.7071096723022171, "['davidgfnet', 'artart78']")	(-0.7071087085567079, "['cadejager', 'jti-lanl', 'gransom']")	(-0.7071086005450822, "['kristiana-rendon', 'thewacokid', 'wfvining', 'jschlind411', 'gmfricke', 'Antonio-Griego', 'akilandrews', 'JCHJones']")	(-0.7071084737556828, "['liesrock', 'alaurenzi', 'duncanjovial', 'nkashiri', 'm-shahbazi']")	(-0.7071082606746788, "['cdondrup', 'mudrole1', 'heuristicus', 'MFernandezCarmona']")	(-0.7071074537397494, "['LoicCadour', 'fcollot', 'msermesant', 'mathildemerle', 'mjuhoor', 'juliencastelneau']")	(-0.7071074537397494, "['fvichot', 'papadop', 'ocommowi']")	(-0.7071070405486111, "['wkrzemien', 'Alvarness', 'alekgajos', 'wictus', 'grey277', 'kkacprzak']")	(-0.6546955705787857, "['donatieng', 'ithinuel', 'studavekar', 'maciejbocianski', 'fkjagodzinski', 'mprse', 'cmonr', 'jamesbeyond', 'hug-dev', 'gaborkertesz', 'shuoo']")	(-0.6546753410409201, "['ahornung', 'christian-rauch', 'felixendres', 'wxmerkt', 'gtinchev']")	(-0.6546660722462896, "['garyservin', 'tulku', 'romainreignier']")	(-0.5773503614349297, "['bobismijnnaam', 'DennisdeW']")	

bluepeppers/archey3	(-0.0019245029830259907, "['jagranata', 'jianli']")	(-0.0017329503371566264, "['KaijenHsiao', 'po1', 'haidai', 'cinchcircuits', 'pbaughman']")	(-0.0016368274410379605, "['tackin', 'vidister']")	(-0.0016361763385786922, "['pchaigno', 'memelet', 'cpliakas', 'awailly', 'paskl47', 'eastokes']")	(-0.00145776836428983, "['NicoHinderling', 'a--hoang', 'lkwatson', 'agoncharova', 'mhendrick98', 'maridematte']")	(-0.001402905814601604, "['KhasanovBI', 'karitra']")	(-0.0011469906847299983, "['sampathP', 'muroi']")	(-0.0011030784224908652, "['zamiam69', 'zsoltn']")	(-0.0009652761934083076, "['DeeJay', 'helenlynn', 'topshed', 'Dan-himself', 'tracygardner', 'jonwitts', 'lawsie', 'jameshay1']")	(-0.0007336628787854428, "['tribeiro', 'wschoenell', 'phsilva', 'ankanaan']")	(-0.000678843411729864, "['bobh0303', 'nrsiward']")	(-0.000596521175619684, "['mfenniak', 'tlocke']")	(-0.0005456251683325647, "['willfurnass', 'ptheywood', 'anthonybrookfield', 'mondus', 'twinkarma']")	(-0.0005442991893096149, "['caohhung', 'martin-ducar-gd']")	(-0.000490602749258152, "['weylin', 'katagatame']")	(-0.00043178184859916035, "['StoneyJackson', 'andrewgalisa', 'collaj']")	(-0.0003852391645631178, "['adamdevigili', 'TrevorBreton']")	(-0.0002961561658852518, "['ldcastell', 'mwturvey']")	(-0.00025593854006429536, "['Exterminus', 'Lucasgscruz']")	(-0.00012358322244106456, "['philadams', 'Lertsenem', 'eeveeta', 'robertpateii', 'synacktic']")	

exynos5410/android_device_samsung_ja3gduosctc	(-0.00010883740991527135, "['viviant100', 'vickymyers', 'loriNakamura']")	(-0.0001087743089087949, "['jiexray', '18822265922']")	(-0.00010649246143563633, "['zactimeinc', 'cjmurphy42']")	(-0.00010550116796546835, "['mtferreira', 'lucasinfoh']")	(-0.00010395205459669672, "['chuan9', 'nevets963', 'wagmarcel', 'arkocal', 'jelyoussefi', 'stevenbriscoe']")	(-9.956272488261969e-05, "['dlozanoarenal', 'rcarballoc']")	(-9.800628166206812e-05, "['jluangphasy', 'cmd-kvn']")	(-9.696898639688602e-05, "['egyptianbman', 'renuzit']")	(-9.215095205495472e-05, "['subdavis', 'cconsidine']")	(-8.75058326734126e-05, "['Gaohaoyang', 'TangMingliang95']")	(-8.601300354695211e-05, "['laurentvaills', 'sauthieg', 'violette']")	(-8.59254728072279e-05, "['samfalconer', 'asmasattar', 'Jeremy-Friedman', 'amk5401']")	(-8.111707212309615e-05, "['wisegb', 'graichen']")	(-8.064581777341048e-05, "['Mortymus', 'ingokru']")	(-8.064581777341048e-05, "['mmoscher', 'da-snap']")	(-7.091499531216674e-05, "['grantcm', 'LauY99', 'aali195']")	(-7.091092859383412e-05, "['Beeert', 'WhiteWolf3500']")	(-6.849134896074689e-05, "['janie-t', 'anna-marie-vos', 'lucas-wills', 'MeghanHughes']")	(-6.368864569836159e-05, "['pengchengan', 'jjamay']")	(-5.7889882789435215e-05, "['brianlmacdonald', 'jvpepino', 'apdjz']")	

Flet/standard-engine	(-0.0012179310824598304, "['bobeagan', 'chrisshiplet', 'chrisheninger']")	(-0.0012117286332342506, "['courtenayt', 'rfriedel']")	(-0.0011701516620898375, "['tkdan235', 'fshost', 'AckerApple']")	(-0.0010355395717727186, "['glenselle', 'yawetse', 'tom5191', 'hortondust']")	(-0.0009467499536113472, "['theoephraim', 'josmcg']")	(-0.0008869639512488896, "['BrunoCartier', 'JohnCoker']")	(-0.0008263830199973958, "['nicolashemonic', 'cruikshj']")	(-0.0008169102585816972, "['tsironis', 'eavgerinos', 'KhaledElAnsari']")	(-0.0007821318140682443, "['Yogu', 'syzer', 'Zertz', 'florianholzapfel', 'dvlsg', 'wesratcliff', 'smirea']")	(-0.000720316672361047, "['s100', 'mattbaileyuk']")	(-0.000705463606009336, "['samaxes', 'AlexDisler', 'tiagorico']")	(-0.0006470734788906655, "['openam', 'mashu-daishi']")	(-0.0006453405420624588, "['pchuri', 'mdravida19']")	(-0.0006222516580790483, "['samuelralak', 'janat08', 'anbud', 'paranoico', 'gazhayes', 'geekyd', 'AshHimself', 'MadanBhandari', 'chamidu-supeshala']")	(-0.0005554724068922053, "['gregra81', 'cyrillef', 'rozman1234', 'zivSher']")	(-0.0004759907520401848, "['benjaminparnell', 'benedfit', 'jackbrewer']")	(-0.0004535517997789263, "['jonpacker', 'NukaPunk']")	(-0.0004353212952628186, "['underscorebrody', 'jintoppy']")	(-0.00020522581422663884, "['brozeph', 'rmomii']")	(-0.00015094436662245909, "['lukasz-wronski', 'hajekj', 'kalieye', 'jsmilovic']")	

potree/PotreeConverter	(-0.7637977990439457, "['jasiek', 'marksteward', 'JasperWallace']")	(-0.7637796225924666, "['quozl', 'AlanJAS', 'vikramahuja1001', 'ezequielpereira', 'devAbnull', 'rdrsadhu', 'dnarvaez', 'cristian99garcia', 'ortegaps', 'zeecoder606', 'vipulgupta2048', 'yashagrawal3', 'octamois', 'shivamchadha', 'pro-panda', 'avinashbharti97']")	(-0.7637748840603187, "['sgress454', 'andyhu', 'vohof', 'boogerlad', 'Karnith', 'gayanhewa', 'thomasdashney', 'delight', 'Rafi993', 'pekpon', 'acekat', 'thecjharries']")	(-0.7637677165442605, "['casallas', 'rpavlik', 'Oberon00', 'vdudouyt', 'janoc', 'sradigan', 'cpldcpu']")	(-0.7637656477620024, "['rdegges', 'jheld', 'sasakalaba', 'tom-smith-okta']")	(-0.7637642265423586, "['kossebau', 'adityab', 'piggz']")	(-0.7637631820871917, "['apiri', 'randerzander', 'jdye64', 'thesolson', 'ottobackwards', 'achristianson']")	(-0.7388121422226158, "['ccrisan', 'jogu', 'jasaw', 'inodlite']")	(-0.7385717004342754, "['scottrobertson', 'marcqualie', 'MatthewBooth']")	(-0.7385628001652176, "['xoarinn', 'Honeybunch', 'MattGuerrette', 'JamesMarcil']")	(-0.7385525176786988, "['bougyman', 'the-maldridge', 'CMB', 'benkasminbullock', 'enguerrand', 'kulinacs', 'masato9000']")	(-0.7385521833477364, "['threeqube', 'cmcavoy', 'samdoshi', 'tehn', 'scanner-darkly', 'burnsauce']")	(-0.7385512126109482, "['taoeffect', 'phelixbtc', 'ryancdotorg', 'shannona']")	(-0.7385500609103987, "['tzlaine', 'ned14', 'ldionne', 'blastrock', 'viboes', 'akrzemi1']")	(-0.7342718308122501, "['xfxyjwf', 'mcos', 'markmandel', 'drobertduke', 'xyzzyz', 'mehrdada']")	(-0.7338037917544675, "['igracia', 'jcaden', 'sancane', 'rbenitez', 'mparis', 'dfernandezlop', 'rvlad', 'pabuhler']")	(-0.6742182034992151, "['samdroid-apps', 'goutamnair7', 'godiard', 'tchx84', 'nemesiscodex', 'inkyfingers', 'FGrose', 'Boquete', 'ezequielpereira']")	(-0.6742074973746109, "['requires', 'dshafik', 'zghosts', 'Webysther']")	(-0.6742005134629507, "['rogueSkib', 'mgh', 'fairfieldt', 'lunarraid', 'yofreke', 'ravitejag', 'AlexTiTanium']")	(-0.6030794949418081, "['Dees-Troy', 'CaptainThrowback', 'Kra1o5', 'bigbiff', 'andi34', 'nailyk-fr', 'djselbeck', 'that1', 'kotzir', 'steadfasterX', 'drmarble', 'ThePiGuy0']")	

ValLaboratory/dokushokai	(-0.000378811278053254, "['qeqar', 'elementyang']")	(-0.00036753209169819445, "['davelcpanelnet', 'naptastic']")	(-0.0003605332501183328, "['RedSquirrelious', 'macrobug', 'pdimitratos', 'zhz2017']")	(-0.0003531994079251676, "['cziegeler', 'jsedding']")	(-0.0003144638524767344, "['nukaga', 'fumikito', 'lunaluna']")	(-0.00031182268525165177, "['prasanthhs', 'guru5590', 'divyasrinivasan1986']")	(-0.0003105457632041812, "['chadbratton', 'sivaOSI', 'brunoOSI', 'pierreOSI', 'VinhOSI', 'amanbop']")	(-0.000294709513271691, "['deadlyicon', 'punitrathore', 'lumodon', 'DianaVashti', 'mKleinCreative', 'TrevorJamesH']")	(-0.0002850909822644533, "['hharnisc', 'hamstu', 'kiriappeee', 'msanroman']")	(-0.00027203600428715046, "['alasarr', 'ignaciodomingo', 'isapozuelo', 'padawannn', 'alejandrohall', 'josemazo', 'Josmorsot', 'dandelea', 'ajaest', 'borjaeguia']")	(-0.0002713260324690977, "['TheYorkWei', 'Lokicity']")	(-0.00023222975324003057, "['nicdoye', 'melahn']")	(-0.0002153564208978166, "['panlinux', 'powersj', 'simpoir']")	(-0.00018324912092260615, "['astolzberg', 'seanpg94']")	(-0.00017669561595915433, "['unamaria', 'stewest']")	(-0.0001472352094057082, "['edgibbs', 'Sushmareddy-OSI', 'eckdanny-osi', 'mlipardo', 'RamuKammagani-osi', 'karthik-sista']")	(-8.185424156409425e-05, "['stefancocora', 'lbennett-hod', 'hod-shcraigen', 'andymarke']")	(-8.006244216133212e-05, "['johntconklin', 'amybuck', 'jghalam', 'hpersh']")	(-7.402273204508341e-05, "['rakeshdatta', 'GarrickHe', 'atanu-mandal', 'theucke']")	(-1.8713788852639835e-05, "['KonstantinRyazantsev', 'AntonSivov', 'andrey-anokhin']")	

nhibberd/mismi	(-0.0001312783975903483, "['callovarne', 'jcowley']")	(-0.00012087104346070295, "['stein321', 'ebracho', 'xiiishaneiiix']")	(-0.00011844895827583785, "['megducharme', 'therealrmac', 'ollieOsh']")	(-0.00011808074365024756, "['diafygi', 'nuxi']")	(-8.825709422057414e-05, "['pengchengan', 'jjamay']")	(-8.124005114154323e-05, "['chuan9', 'nevets963', 'wagmarcel', 'arkocal', 'jelyoussefi', 'stevenbriscoe']")	(-7.811902156100403e-05, "['Beeert', 'WhiteWolf3500']")	(-6.825827506407956e-05, "['wisegb', 'graichen']")	(-6.545011615626537e-05, "['samfalconer', 'asmasattar', 'Jeremy-Friedman', 'amk5401']")	(-6.237215307775596e-05, "['zactimeinc', 'cjmurphy42']")	(-6.0182761683205116e-05, "['jiexray', '18822265922']")	(-5.9060617693407735e-05, "['viviant100', 'vickymyers', 'loriNakamura']")	(-3.9621789792751665e-05, "['jluangphasy', 'cmd-kvn']")	(-3.4344673079180196e-05, "['brianlmacdonald', 'jvpepino', 'apdjz']")	(-2.2810074987114717e-05, "['janie-t', 'anna-marie-vos', 'lucas-wills', 'MeghanHughes']")	(-1.9847686925131236e-05, "['mtferreira', 'lucasinfoh']")	(-1.563513396660081e-05, "['dlozanoarenal', 'rcarballoc']")	(-1.4545316084383865e-05, "['Mortymus', 'ingokru']")	(-1.4545316084383865e-05, "['mmoscher', 'da-snap']")	(-8.851119117580158e-06, "['subdavis', 'cconsidine']")	

bull-mountain/hangoutsbot	(-0.0018161946546974364, "['hynek-urban', 'JirkaChadima']")	(-0.0018128579099316354, "['mwhamgenomics', 'tcezard', 'paalmbj', 'katieemelianova']")	(-0.001697355202701045, "['tebica', 'darjeeling', 'NohSeho', 'channprj', 'woojing', 'kucuny', 'Curzy', 'rikapo', 'iiogmgo', '0x1f440', 'makeajourney']")	(-0.001693575957891479, "['christopherjwang', 'JasonStoltz']")	(-0.0015642764058879778, "['MrSprigster', 'StateOfTheArt89']")	(-0.001536376963000561, "['sinchb', 'taiyangc']")	(-0.0014681394721403097, "['aldreth', 'ninaszymor']")	(-0.0014063311082635444, "['kcratie', 'birdyhuang', 'JaikrishnaTS']")	(-0.0013968221366564663, "['thomasgoirand', 'edmondsw']")	(-0.001163181595287445, "['kstrauser', 'jvantuyl']")	(-0.0010939223422579672, "['tjone270', 'em92']")	(-0.0010429896334004025, "['hansmosh', 'bschnelle', 'kevincrabbe', 'cowleyk', 'karenbowen', 'ssclay']")	(-0.0010429896334004025, "['mattdodge', 'adiazg', 'f1401martin', 'kyliedale', 'jennyknuth', 'tlugger', 'deliciousmonster', 'aaronranard']")	(-0.0010092751167999731, "['tyoungNIO', 'dorzel']")	(-0.0009712571630650181, "['seawolf42', 'costrouc', 'chalmerlowe', '808kimosan', 'JohnJohnsonIII']")	(-0.0007060926412916452, "['mape2k', 'izzy']")	(-0.0005460411356278816, "['lawrancejing', 'mhuin']")	(-0.00041297127399388654, "['willshulman', 'gregbanks']")	(-0.0003562980228712292, "['EmlynC', 'marshallhumble']")	(-0.0002421390574621159, "['im85288', 'hawkeyexp']")	
"""